# An Introduction to Modeling with ModelGraphs.jl
__Jordan Jalving and Victor M. Zavala__ <br>
__University of Wisconsin-Madison__

In [9]:
using Pkg
Pkg.activate("/home/jordan/.julia/dev/ModelGraphs")

"/home/jordan/.julia/dev/ModelGraphs/Project.toml"

### Load Packages

In [10]:
using ModelGraphs
using GLPK
using Ipopt

### Create a ModelGraph 
Here we add one link variable and two master constraints.  
Link variables and master constraints can be thought of as a first stage problem

In [11]:
mg = ModelGraph()

#Master Constraints
@linkvariable(mg,z)
@masterconstraint(mg,z >= 0)
@masterconstraint(mg,z <= 3)

#NOTE 
#It is also possible to say:
# @variable(mg,z)
# @constraint(mg, z >= 0)
# @constraint(mg, z<= 3)
# HOWEVER, this can make the model definition between the graph and its nodes confusing.  We recommend using 
# the macros to facilitate model development.



z - 3 ≤ 0.0

### Add ModelNodes
ModelNode(s) can be treated just like Model(s) from JuMP.  In fact, a ModelNode is a JuMP.AbstractModel extension which allows the straightforward JuMP syntax.

In [12]:
#Add nodes
n1 = add_node!(mg)
@variable(n1,z)
link_variables!(n1[:z],mg[:z])
@variable(n1,x[1:2] >= 1)
@constraint(n1,x[1] + x[2] <= 3)

n2 = add_node!(mg)
@variable(n2,z)
link_variables!(n2[:z],mg[:z])
@variable(n2,x[1:2] >= 0)
@constraint(n2,x[1] + x[2] <= 3)


x[1] + x[2] ≤ 3.0

### Add a link constraint and an overall graph objective
By default, a ModelGraph assumes that the total graph objective as the sum of individual node objectives.  The @graphobjective makes it straightforward to specify various objective functions as a funcion of node and link variables.

In [13]:
#LinkConstraints
@linkconstraint(mg,n1[:x][1] + n2[:x][1] + n1[:x][2] <= 10)

#Set graph objective
@graphobjective(mg,Min,n1[:x][1])


x[1]

### We can aggregate a ModelGraph into a single AggregateModel object
Aggregation facilitates all of the ModelGraph solver interfaces.  A ModelGraph can be aggregated completely as shown here to use the solvers that interface with JuMP and MOI, or it can be aggregated in such a way to create block-structures for distributed solvers.

In [14]:
agg_model,aggregation_map = aggregate(mg)
ipopt = Ipopt.Optimizer
optimize!(agg_model,ipopt)

This is Ipopt version trunk, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        9
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        5
                     variables with only lower bounds:        4
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        5
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        4

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0100000e+00 0.00e+00 7.62e-01  -1.0 0.00e+00    -  0.00e+00 0.00e+00   0
   1 

### If we call optimize! on a ModelGraph with a JuMP solver object, then it will solve the aggregated model and populate the ModelGraph solution.

In [15]:
glpk = GLPK.Optimizer
optimize!(mg,glpk)

println(nodevalue.(n1[:x]))
println(nodevalue.(n2[:x]))

Aggregating Model Graph...
Optimizing Aggregated Model...
Found Solution
[1.0, 1.0]
[0.0, 0.0]


┌ Warning: Constant in scalar function moved into set.
└ @ LinQuadOptInterface /home/jordan/.julia/packages/LinQuadOptInterface/ZMx9f/src/constraints/scalaraffine.jl:146


In [ ]:
#OTHER MACROS
#@NLmasterconstraint (works)
#@NLlinkconstraint (almost works)
#@NLnodeconstraint (works)
#@NLgraphobjective (almost works)
